## CLIPSPY clinical decision support rules

### Developing CDS rules to data extracted using a FHIR server

#### Note: To be done AFTER the hands-on tutorial and the tutorial on rule programming

#### Requirements

1. install into the same Python environment as clipspy the Python FHIR client: https://github.com/dbmi-pitt/client-py.git 

1.1. download the file https://github.com/dbmi-pitt/client-py/blob/master/fhirclient-4.0.0.tar.gz

1.2. change to the folder where you saved the file and use pip from the command line: pip install fhirclient-4.0.0.tar.gz
 

2. xmljson: https://pypi.org/project/xmljson/ (should install easily with pip: pip install xmljson

3. the Pitt VPN running with the DBMI role

4. confirm that the FHIR server is accessible and running by going to this URL in your browser:

http://130.49.206.139:8080/omoponfhir-stu3/tester/ 


#### E6: Exercise writing and applying production rules 

Your task is to implement the drug interaction CDS algorithm described at https://ddi-cds.org/warfarin-antidepressants/. You will test your rule-based implementation of the algorithm by running it over facts that you create from the Patient, MedicationPlan, and Condition FHIR resources queried above. The value sets for all of the drugs and conditions used in the algorithm are loaded into working memory using the blocks above. Your job is to:

1. Develop templates for the three key knowledge entities (patients, medications, and conditions). You do not need to represent all FHIR resource properties, only those that matter for the algorithm  

2. Develop rules that implement the logic diagram shown on the web page so that each branch of the rule is covered. Aim for your rules to be as modular as possible. It is better for you to create allot of very simple rules than try to create a small number of relatively complex rules. Let the reasoner do most of the work. 

3. Load the entire dataset retrieved in the blocks above into CLIPS working memory as facts that use your templates. Use Python to translate from the FHIR resources to strings representing facts, and then load those facts into the CLIPS environment.

4. Run the rule engine so that it produces an operational classification with respect to the drug interaction algorithm (leaf nodes in the diagram) for each patient. Show the patients that are exposed to the interacting drugs and what oparational classification the algorithm would assign them.


NOTE: All visits, drugs, conditions start on 2/13/2008 and end on 4/27/2008. Also, there are 7 patients that should trigger a rule branch

NOTE: I suggest that you append log information to the global variable ?\*log\* because STDERR and STDOUT do not work like expected in the CLIPS environment.  


#### Load all of the required value sets as facts

In [1]:
# libraries needed to load values sets as XML and convert to JSON
import xmljson
from json import dumps
from xmljson import yahoo
from xml.etree.ElementTree import fromstring

In [2]:
def getConceptCodes(vs):
    return vs['{urn:ihe:iti:svs:2008}RetrieveValueSetResponse']['{urn:ihe:iti:svs:2008}ValueSet']['{urn:ihe:iti:svs:2008}ConceptList']['{urn:ihe:iti:svs:2008}Concept']


In [3]:
## Set up the CLIPS environment
import clips
env = clips.Environment()

In [4]:
# Clear working memory
env.clear()
env.reset()

# Simple template for a value set
s = """
 (deftemplate valueset
    (slot vs_name)
    (slot code)
    (slot label))
"""
env.build(s)

In [5]:
# aldosterone-antagonists
f = open('tutorial-data/value-sets/aldosterone-antagonist-vs.xml')
buf = f.read()
f.close()
aldosterone_antagonist = yahoo.data(fromstring(buf))
ccs = getConceptCodes(aldosterone_antagonist)
for cc in ccs:
    env.assert_string('''
(valueset
   (vs_name "{}")
   (code {})
   (label "{}")
)        
    '''.format('altosterone-antagonists',cc['code'],cc['displayName']))
    
# bupropions    
f = open('tutorial-data/value-sets/bupropion-vs.xml')
buf = f.read()
f.close()
bupropion = yahoo.data(fromstring(buf))
ccs = getConceptCodes(bupropion)
for cc in ccs:
    env.assert_string('''
(valueset
   (vs_name "{}")
   (code {})
   (label "{}")
)        
    '''.format('bupropion',cc['code'],cc['displayName']))
   
# mirtazapine
f = open('tutorial-data/value-sets/mirtazapine-vs.xml')
buf = f.read()
f.close()
mirtazapine = yahoo.data(fromstring(buf))
ccs = getConceptCodes(mirtazapine)
for cc in ccs:
    env.assert_string('''
(valueset
   (vs_name "{}")
   (code {})
   (label "{}")
)        
    '''.format('mirtazapine',cc['code'],cc['displayName']))

# nsaids
f = open('tutorial-data/value-sets/NSAIDs-vs.xml')
buf = f.read()
f.close()
nsaids = yahoo.data(fromstring(buf))
ccs = getConceptCodes(nsaids)
for cc in ccs:
    env.assert_string('''
(valueset
   (vs_name "{}")
   (code {})
   (label "{}")
)        
    '''.format('nsaids',cc['code'],cc['displayName']))


# SSRIs and SNRIs
f = open('tutorial-data/value-sets/SSRI-SNRI-vs.xml')
buf = f.read()
f.close()
SSRI_SNRI = yahoo.data(fromstring(buf))
ccs = getConceptCodes(SSRI_SNRI)
for cc in ccs:
    env.assert_string('''
(valueset
   (vs_name "{}")
   (code {})
   (label "{}")
)        
    '''.format('SSRI_SNRI',cc['code'],cc['displayName']))

# systemic_corticosteriods
f = open('tutorial-data/value-sets/systemic-corticosteriods-vs.xml')
buf = f.read()
f.close()
systemic_corticosteriods = yahoo.data(fromstring(buf))
ccs = getConceptCodes(systemic_corticosteriods)
for cc in ccs:
    env.assert_string('''
(valueset
   (vs_name "{}")
   (code {})
   (label "{}")
)        
    '''.format('systemic_corticosteriods',cc['code'],cc['displayName']))

# tricyclic_antidepr
f = open('tutorial-data/value-sets/tricyclic-antidepressants.xml')
buf = f.read()
f.close()
tricyclic_antidepressants = yahoo.data(fromstring(buf))
ccs = getConceptCodes(tricyclic_antidepressants)
for cc in ccs:
    env.assert_string('''
(valueset
   (vs_name "{}")
   (code {})
   (label "{}")
)        
    '''.format('tricyclic_antidepressants',cc['code'],cc['displayName']))

# warfarin
f = open('tutorial-data/value-sets/warfarin-vs.xml')
buf = f.read()
f.close()
warfarin = yahoo.data(fromstring(buf))
ccs = getConceptCodes(warfarin)
for cc in ccs:
    env.assert_string('''
(valueset
   (vs_name "{}")
   (code {})
   (label "{}")
)        
    '''.format('warfarin',cc['code'],cc['displayName']))

for fact in env.facts():
    print(fact)

(initial-fact)
(valueset (vs_name "altosterone-antagonists") (code 104230) (label "Spironolactone 1 MG/ML Oral Suspension"))
(valueset (vs_name "altosterone-antagonists") (code 104231) (label "Spironolactone 2 MG/ML Oral Suspension"))
(valueset (vs_name "altosterone-antagonists") (code 104232) (label "Spironolactone 5 MG/ML Oral Suspension"))
(valueset (vs_name "altosterone-antagonists") (code 104233) (label "Spironolactone 10 MG/ML Oral Suspension"))
(valueset (vs_name "altosterone-antagonists") (code 198222) (label "Spironolactone 100 MG Oral Tablet"))
(valueset (vs_name "altosterone-antagonists") (code 198223) (label "Spironolactone 50 MG Oral Tablet"))
(valueset (vs_name "altosterone-antagonists") (code 198224) (label "Hydrochlorothiazide 25 MG / Spironolactone 25 MG Oral Tablet"))
(valueset (vs_name "altosterone-antagonists") (code 198225) (label "Hydrochlorothiazide 50 MG / Spironolactone 50 MG Oral Tablet"))
(valueset (vs_name "altosterone-antagonists") (code 246399) (label "Fur

#### Obtain all Patient, MedicationStatement, and Condition resources in FHIR STU3 

NOTE: You can browse the FHIR data directly at http://130.49.206.139:8080/omoponfhir-stu3/fhir (NOTE: You HAVE to be logged into the Pitt VPN with the correct DBMI role)

In [6]:
# Using https://github.com/dbmi-pitt/client-py.git for FHIR STU3 
# and our own synthetic data on AWS
from fhirclient import client
settings = {
    'app_id':'no matter',
    'api_base': 'http://130.49.206.139:8080/omoponfhir-stu3/fhir/'
}
syn_omop = client.FHIRClient(settings=settings)

# This global variable is where we write statements we want to see printed. We can print the value of the 
# variable out when we want to view it
env.build('(defglobal ?*log* = (format nil "INFO:%n"))')

In [7]:
import fhirclient.models.patient as patient_state
search = patient_state.Patient.where(struct={})
patient_states = search.perform_resources(syn_omop.server)
print("{} Patient states".format(len(patient_states)))
print("Example:\n\t{}".format(patient_states[0].as_json()))

79 Patient states
Example:
	{'id': '1495', 'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']}, 'extension': [{'extension': [{'url': 'ombCategory', 'valueCoding': {'code': '2106-3', 'display': 'White', 'system': 'urn:oid:2.16.840.1.113883.6.238'}}], 'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'}, {'extension': [{'url': 'ombCategory', 'valueCoding': {'code': '2186-5', 'display': 'Non Hispanic or Latino', 'system': 'urn:oid:2.16.840.1.113883.6.238'}}], 'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'}], 'text': {'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText"></div><table class="hapiPropertyTable"><tbody><tr><td>Address</td><td><span>null </span><br/><span>null </span><br/></td></tr><tr><td>Date of birth</td><td><span>01 January 1935</span></td></tr></tbody></table></div>', 'status': 'generated'}, 'active': False, 'address': [{'use': 'home'}], 'birthDate': '1935-01-01', 'g

In [8]:
import fhirclient.models.medicationstatement as mstate
search = mstate.MedicationStatement.where(struct={})
medstates = search.perform_resources(syn_omop.server)
print("{} MedicationPlan states".format(len(medstates)))
print("Example:\n\t{}".format(medstates[0].as_json()))

100 MedicationPlan states
Example:
	{'id': '160000', 'dosage': [{'text': '3 times daily'}], 'effectivePeriod': {'end': '2008-04-27T00:00:00+00:00', 'start': '2008-02-13T00:00:00+00:00'}, 'medicationCodeableConcept': {'coding': [{'code': '855296', 'display': 'Warfarin Sodium 10 MG Oral Tablet', 'system': 'http://www.nlm.nih.gov/research/umls/rxnorm'}]}, 'subject': {'reference': 'Patient/1495'}, 'resourceType': 'MedicationStatement'}


In [9]:
import fhirclient.models.condition as condstate
search = condstate.Condition.where(struct={})
condstates = search.perform_resources(syn_omop.server)
print("{} Condition states".format(len(condstates)))
print("Example:\n\t{}".format(condstates[0].as_json()))

8 Condition states
Example:
	{'id': '140000', 'abatementDateTime': '2008-04-27T00:00:00+00:00', 'category': [{'coding': [{'code': 'OMOP generated', 'display': 'EHR encounter diagnosis', 'system': 'None'}]}], 'code': {'coding': [{'code': '12274003', 'display': 'Acute peptic ulcer with hemorrhage', 'system': 'http://snomed.info/sct'}]}, 'onsetDateTime': '2008-02-13T00:00:00+00:00', 'subject': {'reference': 'Patient/1496'}, 'resourceType': 'Condition'}
